In [1]:
import cx_Oracle

usr ='movies_project'
pwd = 'intekglobal123'

#tns="db:1521/xe"
dsn = cx_Oracle.makedsn("db", 1521, service_name="xepdb1")

connection = cx_Oracle.connect(usr, pwd, dsn,encoding="UTF-8")


#check connectivity
cursor = connection.cursor()
for val in cursor.execute("""   SELECT * FROM DUAL """):
    print("Values:", val)

Values: ('X',)


## FUNCTIONS

In [2]:
def q_max_names(colname):
    '''
    Select max number of names in all columns.
    Input: Column Name
    Output: Query String
    '''
    return f"""  
SELECT MAX(num_{colname}) FROM  (
SELECT  (LENGTH( "{colname}" ) - LENGTH(REPLACE( "{colname}" ,',','')) + 1) AS num_{colname} 
FROM (
SELECT TO_CHAR(TRIM("{colname}" )) AS "{colname}"   FROM ROTTEN_TOMATOES_RAW rtr 
WHERE "{colname}"  IS NOT NULL 
)
)
"""

#example
max_writers =  [int(result[0]) for col in ['WRITER'] for result in cursor.execute(q_max_names(col))][0]

max_writers

8

In [3]:
def union_all(name_prefix,new_name,max_columns):
    """ union all columns {name_prefix}_1, ... , {name_prefix}_{max_columns} in a {new_name}
        Incorporates film_name
    """
    selec_from =  lambda k: f"""SELECT rtf.*, {name_prefix}_{k} AS {new_name} FROM ROTTEN_TOMATOES_FLAT rtf
    WHERE {name_prefix}_{k} is not null
    """
    q_string = '\n UNION ALL \n'.join([selec_from(k) for k in range(1,max_columns+1)])
    return q_string 

def union_and_equal_value(name_prefix,new_name,max_columns,col_value):
    """ applies union_all(...) and 'where' equal to {col_value} clause
    """
    return f"""
SELECT {new_name}_db.*
FROM ({union_all(name_prefix,new_name,max_columns)}) {new_name}_db

WHERE {new_name}_db.{new_name} = '{col_value}'
"""

## QUERIES

### One
> Which was the highest/lowest-grossing movie for the available years? 	

In [4]:
gross_num = f"""
SELECT rtr.*, 
    CASE SUBSTR(bogu,-1) 
        WHEN 'M' THEN TO_NUMBER(REVERSE(SUBSTR(REVERSE(bogu),2))) * 1000000
        WHEN 'K' THEN TO_NUMBER(REVERSE(SUBSTR(REVERSE(bogu),2))) * 1000
        ELSE NULL
    END AS GROSS_NUM
FROM 
    (
    SELECT rtr.*, 
           SUBSTR(TO_CHAR( BOX_OFFICE_GROSS_USA),2) AS bogu 
    FROM ROTTEN_TOMATOES_RAW rtr
    ) rtr
""" 

In [5]:

for k,v in zip(['highest','lowest'],cursor.execute(q_1)):
    
    print(k,'film:',v[0], '-- box_office_gross', str(v[1]) +' Dlls.')

highest film: Up for Grabs (2005) -- box_office_gross 1900 Dlls.
lowest film: Avengers: Endgame (2019) -- box_office_gross 858400000 Dlls.


### Two
  > Which are the movies directed by George Lucas? 	

In [6]:
max_writers =  [int(result[0]) for col in ['WRITER'] for result in cursor.execute(q_max_names(col))][0]

max_writers

8

In [7]:
for r in  cursor.execute(union_and_equal_value('writer','n_writer',max_writers,'George Lucas')):
    print(r[0],r[1])

What about Woody Allen?

In [8]:
for r in  cursor.execute(union_and_equal_value('writer','n_writer',max_writers,'Woody Allen')):
    print(r[0],r[1])

2011 Midnight in Paris (2011)
2013 Blue Jasmine (2013)


### Three

 > List the different genres available in the database

In [9]:
max_genres =  [int(result[0]) for col in ['GENRE'] for result in cursor.execute(q_max_names(col))][0]

max_genres

6

In [10]:
union_genres_union_clauses = union_all(name_prefix='genre',new_name ='n_genre',max_columns=max_genres)
union_genres = f"""
SELECT DISTINCT(n_genre)
FROM ({union_genres_union_clauses})
ORDER BY n_genre
"""
print(union_genres)


SELECT DISTINCT(n_genre)
FROM (SELECT rtf.*, genre_1 AS n_genre FROM ROTTEN_TOMATOES_FLAT rtf
    WHERE genre_1 is not null
    
 UNION ALL 
SELECT rtf.*, genre_2 AS n_genre FROM ROTTEN_TOMATOES_FLAT rtf
    WHERE genre_2 is not null
    
 UNION ALL 
SELECT rtf.*, genre_3 AS n_genre FROM ROTTEN_TOMATOES_FLAT rtf
    WHERE genre_3 is not null
    
 UNION ALL 
SELECT rtf.*, genre_4 AS n_genre FROM ROTTEN_TOMATOES_FLAT rtf
    WHERE genre_4 is not null
    
 UNION ALL 
SELECT rtf.*, genre_5 AS n_genre FROM ROTTEN_TOMATOES_FLAT rtf
    WHERE genre_5 is not null
    
 UNION ALL 
SELECT rtf.*, genre_6 AS n_genre FROM ROTTEN_TOMATOES_FLAT rtf
    WHERE genre_6 is not null
    )
ORDER BY n_genre



In [11]:
for r in  cursor.execute(union_genres):
    print(r)

('Action',)
('Adventure',)
('Animation',)
('Anime',)
('Biography',)
('Comedy',)
('Crime',)
('Documentary',)
('Drama',)
('Fantasy',)
('History',)
('Horror',)
('Kids And Family',)
('Music',)
('Musical',)
('Mystery And Thriller',)
('Other',)
('Romance',)
('Sci Fi',)
('Sports And Fitness',)
('War',)
('Western',)


### Four
> Which genre has the highest rating on average?

In [12]:
union_genres_union_clauses = union_all(name_prefix='genre',new_name ='n_genre',max_columns=max_genres)
genre_and_score = f"""
SELECT 
n_genre AS genre,
CASE
	WHEN LENGTH(TRIM(TO_CHAR(TOMATOMETER_SCORE))) < 4
		THEN TO_NUMBER( 
		          REVERSE(SUBSTR(
		          REVERSE(
		                 TRIM(TO_CHAR(TOMATOMETER_SCORE))
		                 )
		          ,2)) 
		          )  
END SCORE 
FROM ({union_genres_union_clauses}) 
"""

avg_score_per_genre =  f"""
 SELECT genre, AVG(score) as mean
FROM ({genre_and_score})
GROUP BY genre 

"""

max_avg_score_per_genre =  f"""
 SELECT MAX(mean)
 FROM ({avg_score_per_genre})
 """

genres_with_highest_score = f"""
SELECT *
FROM ({avg_score_per_genre})
WHERE mean = ({max_avg_score_per_genre})
"""

print(' Average Score per Genre \n')
for r in  cursor.execute(avg_score_per_genre):
    print(r)


print('\n Max Value of the Average Score per Genre \n')
    
for r in  cursor.execute(max_avg_score_per_genre):
    print(r)

    
print('\n Genre(s) with highest score on average \n')
    
for r in  cursor.execute(genres_with_highest_score):
    print(r)

 Average Score per Genre 

('Documentary', 93.35860058309038)
('Kids And Family', 88.32876712328768)
('Mystery And Thriller', 89.12946428571429)
('Adventure', 88.45454545454545)
('Fantasy', 88.1842105263158)
('Romance', 88.40336134453781)
('Sports And Fitness', 88.75)
('Horror', 90.62121212121212)
('Sci Fi', 86.82191780821918)
('Animation', 90.54794520547945)
('Action', 87.04379562043796)
('History', 91.34736842105264)
('Other', 90.33333333333333)
('Biography', 90.24509803921569)
('Anime', 90.33333333333333)
('Drama', 90.14988290398126)
('Comedy', 89.2463768115942)
('Crime', 89.60215053763442)
('War', 90.4)
('Western', 90.55555555555556)
('Music', 92.1025641025641)
('Musical', 89.16666666666667)

 Max Value of the Average Score per Genre 

(93.35860058309038,)

 Genre(s) with highest score on average 

('Documentary', 93.35860058309038)


### FIVE
> Which is the least profitable genre (on average)?

In [13]:
union_genres_union_clauses = union_all(
    name_prefix='genre', new_name='n_genre', max_columns=max_genres
)

genre_and_gross = f"""
SELECT 
n_genre AS genre, gross_num
FROM (
    SELECT rtf.*, 
        CASE SUBSTR(bogu,-1) 
            WHEN 'M' THEN TO_NUMBER(REVERSE(SUBSTR(REVERSE(bogu),2))) * 1000000
            WHEN 'K' THEN TO_NUMBER(REVERSE(SUBSTR(REVERSE(bogu),2))) * 1000
            ELSE NULL
        END AS GROSS_NUM
    FROM 
        (
        SELECT rtf.*, SUBSTR(TO_CHAR( BOX_OFFICE_GROSS_USA),2) AS bogu FROM ({union_genres_union_clauses})  rtf
       ) rtf
)       
"""

avg_gross_per_genre = f"""
 SELECT genre, AVG(gross_num) as mean
FROM ({genre_and_gross})
GROUP BY genre 

"""

max_avg_gross_per_genre = f"""
 SELECT MAX(mean)
 FROM ({avg_gross_per_genre})
 """

genres_with_highest_gross = f"""
SELECT *
FROM ({avg_gross_per_genre})
WHERE mean = ({max_avg_gross_per_genre})
"""

print(' Average Gross Number per Genre \n')
for r in cursor.execute(avg_gross_per_genre):
    print(r)

print('\n Max Value of the Average Gross Number per Genre \n')

for r in cursor.execute(max_avg_gross_per_genre):
    print(r)

print('\n Genre(s) with highest Gross Number on average \n')

for r in cursor.execute(genres_with_highest_gross):
    print(r)

 Average Gross Number per Genre 

('Documentary', 2504043.8311688313)
('Kids And Family', 148341546.55172414)
('Mystery And Thriller', 37955485)
('Adventure', 178596776.05633804)
('Fantasy', 188697955)
('Romance', 23522772.897196263)
('Sports And Fitness', 6205350)
('Horror', 29428121.42857143)
('Sci Fi', 169652024.24242425)
('Animation', 130001068.51851852)
('Action', 163610438.70967743)
('History', 25729268.888888888)
('Other', 501333.3333333333)
('Biography', 25255140.425531916)
('Anime', 5959350)
('Drama', 18136903.641092326)
('Comedy', 54064438.666666664)
('Crime', 23727787.777777776)
('War', 19913485.36585366)
('Western', 41612193.333333336)
('Music', 25433709.09090909)
('Musical', 122544385.71428572)

 Max Value of the Average Gross Number per Genre 

(188697955,)

 Genre(s) with highest Gross Number on average 

('Fantasy', 188697955)


### SIX
 > Which are the top  movie's original languages?

In [14]:
order_language = f"""
SELECT language, COUNT(language)
FROM (
     SELECT TO_CHAR(original_language) AS language
     FROM ROTTEN_TOMATOES_RAW 
     )
GROUP BY language
ORDER BY COUNT(language) DESC
"""

for r in cursor.execute(order_language):
    print(r)

('English', 1139)
('French (France)', 49)
('French (Canada)', 46)
('Spanish', 38)
('English (United Kingdom)', 36)
('Japanese', 27)
('German', 23)
('Chinese', 20)
('Arabic', 19)
('Hebrew', 14)
('Persian', 13)
('Korean', 13)
('Portuguese (Brazil)', 11)
('Spanish (Spain)', 10)
('Italian', 10)
('Danish', 9)
('Swedish', 9)
('Romanian', 9)
('Russian', 8)
('Turkish', 7)
('Norwegian', 7)
('Polish', 6)
('English (Australia)', 5)
('Thai', 4)
('Hindi', 4)
('Indonesian', 4)
('Hungarian', 4)
('Finnish', 4)
('Icelandic', 3)
('Dutch', 3)
('Georgian', 2)
('Marathi', 2)
('Nepali', 2)
('Unknown language', 2)
('Portuguese', 2)
('Albanian', 1)
('Czech', 1)
('Yiddish', 1)
('Tibetan', 1)
('Lingala', 1)
('Tagalog', 1)
('Mongolian', 1)
('Catalan', 1)
('Afrikaans', 1)
('Lao', 1)
('Dzongkha', 1)
('Bambara', 1)
('Serbian', 1)
('Greek', 1)
('Kalaallisut', 1)
('Kurdish', 1)
('Wolof', 1)
(None, 0)


### SEVEN
> Which movie has more positive/negative critics? What are the genres?      


### EIGTH
> Which director has directed more movies?

In [15]:
max_directors =  [int(result[0]) for col in ['DIRECTOR'] for result in cursor.execute(q_max_names(col))][0]

max_directors

5

In [16]:
union_directors_union_clauses = union_all(name_prefix='director',new_name ='n_director',max_columns=max_directors)

In [17]:
director_freq = f"""
SELECT  n_director AS director, COUNT(n_director) AS num_movies
FROM ({union_directors_union_clauses})
GROUP BY n_director
"""

max_director = f"""
 SELECT MAX(num_movies)
 FROM ({director_freq})
 """

directors_with_highest_num_films = f"""
SELECT *
FROM ({director_freq})
WHERE num_movies = ({max_director})
"""

print('\n Max number of films  directed by one person \n')

for r in cursor.execute(max_director):
    print(r)

print('\n Director(s) with the highest number of films \n')

for r in cursor.execute(directors_with_highest_num_films):
    print(r)


 Max number of films  directed by one person 

(11,)

 Director(s) with the highest number of films 

('Alex Gibney', 11)


### NINE
> Which movies have a score of 9 or higher? Who directed them?  
> What was the average budget for movies with a score/rating of 90 or higher?  

In [18]:
percentage_to_num = lambda col_name: f"""
   CASE
    WHEN LENGTH(TRIM(TO_CHAR({col_name}))) < 4
    THEN TO_NUMBER( 
              REVERSE(SUBSTR(
              REVERSE(
                     TRIM(TO_CHAR({col_name}))
                     )
              ,2)) 
              )  
    END {col_name}
"""

tomatometer_score = percentage_to_num('tomatometer_score')
audience_score = percentage_to_num('audience_score')

scores_to_num = f"""
SELECT film_name, director,
 {tomatometer_score},
 {audience_score}
 FROM ROTTEN_TOMATOES_RAW rtr
"""

tomatoes_greater_than_99 = f"""
SELECT *
FROM ({scores_to_num})
WHERE tomatometer_score >= 99
"""

audience_greater_than_95 = f"""
SELECT *
FROM ({scores_to_num})
WHERE audience_score >= 95
"""

ave_audience_score_for_greater_than_95 = f"""
SELECT *
FROM ({scores_to_num})
WHERE audience_score >= 95
"""


print("Tomatometer Score greater than 99 percent\n")

for r in cursor.execute(tomatoes_greater_than_99):
    print(r[0], r[1], r[2])

print("\n Audience Score greater than 95  percent\n")

for r in cursor.execute(audience_greater_than_95):
    print(r[0], r[1], r[3])
    
print("\n Average buget for movies with  95  percent(audience score)\n")

for r in cursor.execute(audience_greater_than_95):
    print(r[0], r[1], r[3])

Tomatometer Score greater than 99 percent

A Separation (2011) Asghar Farhadi 99
Le Havre (2011) Aki Kaurismäki 99
The Interrupters (2011) Steve James 99
Bill Cunningham New York (2011) Richard Press 99
Cameraperson (2016) Kirsten Johnson 99
Tower (2016) Keith Maitland 99
Under The Shadow (2016) Babak Anvari 99
Things to Come (L'avenir) (2016) Mia Hansen-Løve 99
The Wailing (Goksung) (2016) Na Hong-jin 99
Selma (2014) Ava DuVernay 99
Starred Up (2014) David Mackenzie 99
The Missing Picture (2014) Rithy Panh 99
Elaine Stritch: Shoot Me (2014) Chiemi Karasawa 99
Song Of The Sea (2014) Tomm Moore 99
Gloria (2014) Sebastián Lelio 99
How to Train Your Dragon (2010) Christopher Sanders, Dean DeBlois 99
The Invisible War (2012) Kirby Dick 99
Apollo 11 (2019) Todd Douglas Miller 99
Amazing Grace (2019) Alan Elliott 99
Ash Is Purest White (2019) ZhangKe Jia 99
They Shall Not Grow Old (2019) Peter Jackson 99
Knock Down the House (2019) Rachel Lears 99
For Sama (2019) Edward Watts, Waad Al-Khatea

In [19]:
gross_to_num = f"""
(
    SELECT rtf.*, 
        CASE SUBSTR(bogu,-1) 
            WHEN 'M' THEN TO_NUMBER(REVERSE(SUBSTR(REVERSE(bogu),2))) * 1000000
            WHEN 'K' THEN TO_NUMBER(REVERSE(SUBSTR(REVERSE(bogu),2))) * 1000
            ELSE NULL
        END AS GROSS_NUM
    FROM 
        (
        SELECT rtf.*, SUBSTR(TO_CHAR( BOX_OFFICE_GROSS_USA),2) AS bogu FROM ROTTEN_TOMATOES_RAW  rtf
       ) rtf
)       
"""

percentage_to_num = lambda col_name: f"""
   CASE
    WHEN LENGTH(TRIM(TO_CHAR({col_name}))) < 4
    THEN TO_NUMBER( 
              REVERSE(SUBSTR(
              REVERSE(
                     TRIM(TO_CHAR({col_name}))
                     )
              ,2)) 
              )  
    END {col_name}
"""



tomatometer_score = percentage_to_num('tomatometer_score')
audience_score = percentage_to_num('audience_score')

scores_to_num = f"""
SELECT gross_num,
 {tomatometer_score},
 {audience_score}
 FROM ({gross_to_num})
"""


ave_audience_score_for_greater_than_95 = f"""
SELECT AVG(gross_num)
FROM (SELECT * FROM ({scores_to_num}) WHERE audience_score >= 95)
"""

    
print("\n Average buget for movies with  95  percent(audience score)\n")

for r in cursor.execute(ave_audience_score_for_greater_than_95):
    print(r)


 Average buget for movies with  95  percent(audience score)

(43629906.666666664,)


### TEN

> What is the movie with the highest score per year?  

In [30]:
f"""

FROM (SELECT * rotten_tomatoes_raw)
GROUP BY 
"""

percentage_to_num = lambda col_name: f"""
   CASE
    WHEN LENGTH(TRIM(TO_CHAR({col_name}))) < 4
    THEN TO_NUMBER(
            REVERSE(
              SUBSTR(
                REVERSE(TRIM(TO_CHAR({col_name})))
              ,2)
            ) 
         )  
    END {col_name}
"""

tomatometer_score = percentage_to_num('tomatometer_score')
audience_score = percentage_to_num('audience_score')

year_of_release = f"""
  SELECT rtr.*, EXTRACT( YEAR FROM TO_DATE(RELEASE_DATE_STREAMING,'MONTH DD, YYYY')) AS year_of_release
  FROM ROTTEN_TOMATOES_RAW rtr
"""

scores_to_num = f"""
SELECT film_name, year_of_release,
 {tomatometer_score},
 {audience_score}
FROM ({year_of_release}) 
"""

print("\n Average buget for movies with  95  percent(audience score)\n")

for r in cursor.execute(scores_to_num):
    print(r)


 Average buget for movies with  95  percent(audience score)

(<cx_Oracle.LOB object at 0x7f87efadc450>, 2014, 96, 89)
(<cx_Oracle.LOB object at 0x7f87efadc510>, 2014, 95, 87)
(<cx_Oracle.LOB object at 0x7f87efadc450>, 2014, 99, 92)
(<cx_Oracle.LOB object at 0x7f87efadc510>, 2013, 92, 79)
(<cx_Oracle.LOB object at 0x7f87efadc450>, 2015, 92, 82)
(<cx_Oracle.LOB object at 0x7f87efadc510>, 2013, 94, 86)
(<cx_Oracle.LOB object at 0x7f87efadc450>, 2015, 95, 80)
(<cx_Oracle.LOB object at 0x7f87efadc510>, 2014, 94, 76)
(<cx_Oracle.LOB object at 0x7f87efadc450>, 2016, None, 86)
(<cx_Oracle.LOB object at 0x7f87efadc510>, 2016, 99, 75)
(<cx_Oracle.LOB object at 0x7f87efadc450>, 2016, 99, 82)
(<cx_Oracle.LOB object at 0x7f87efadc510>, 2016, 97, 84)
(<cx_Oracle.LOB object at 0x7f87efadc450>, 2016, None, 84)
(<cx_Oracle.LOB object at 0x7f87efadc510>, 2015, None, 85)
(<cx_Oracle.LOB object at 0x7f87efadc450>, 2016, 99, 92)
(<cx_Oracle.LOB object at 0x7f87efadc510>, 2017, None, 86)
(<cx_Oracle.LOB ob

In [21]:
	    
Find the movie with fewer awards from the top 20 most expensive movies. (Show output as “movie-x won N awards and had a budget of $99,999,999.00” )
Find the movie with the most awards from the 20 less expensive movies.
Name actors performing in ‘The godfather’						
Which actor/actress has appeared in more movies?		
How much money was spent making movies in the year 1991?
What is the average running time per decade?
Create a column where the legend "OSCAR" appears if the movies have won any Oscars and the legend "no-oscar" if none. show the list in alphabetical order
Guess the name of the actor appearing in the list of movies! Create a view that returns 
the first and last string of the actor's name and "*****" in between. 
How often critics disagree with the fan’s opinion?	

SyntaxError: invalid syntax (<ipython-input-21-2753b6222cad>, line 1)

In [ ]:
q ="""
SELECT table_name, column_name, data_type, data_length
FROM USER_TAB_COLUMNS
WHERE table_name = 'ROTTEN_TOMATOES_RAW'
"""
for r in  cursor.execute(q):
    print(r)